## Chapter 3: Pods: Running Containers in Kubernetes

* Pods
    * Co-located group of containers
    * Basic building block of K8s
    * Pods can contain multiple nodes, but pods can never spand multiple worker nodes

## Kubectl commands
* kubectl get {resource} -o yaml: Gets yaml manifest for a given resource
* kubectl explain {resource}: Gets documentation for a given resource
* kubectl create -f {file_name}: Creates a resource from a yaml or json file
* kubectl logs {resource_name}: Gets a resource's logs
* kubectl port-forward {resource_name} {local_port}:{tgt_port}: Allow for external pod connecting
* kubectl label {resource} {name} {key}={value}: Label a resource with a tag
* kubectl get {resource} --show-labels: Lists labels associated with a resources
* kubectl get {resources} -l {key}={value}: Get resource with an associated label key/value
* kubectl get ns: Gets all namesapces in the cluster
* kubectl get pods --namespace {value}: Gets pods of a given namespace
  * Also -n argument
* kubectl create namespace {name}: Creates a namespace
* kubectl create {resource} -n {name}: Creates a resource in a specified namespace
* kubectl delete {resource} {name}: Deletes a specified resource with a specified name

### Why have multiple containers for multiple processes? 
* Containers are designed to run only a single process (or multiple child processes) per container
* Would be difficult to parse through the logs
* Would be difficult to restart one process if it fails
* Pods then allow for the binding/management of containers as a single unit
* Pods allow for you to run closely related processes with the ~exact environments

* All pods in a cluster reside in a single flat, shared, network-address space
    * Every pod can access every other pod at the other pod's IP address
    * No Network Address Translation (NAT) exists between pods
        * Similiar communication to a Local Area Network (LAN)

* Multiple pods is great for a few reasons:
    * Although it's *not illegal* to run a multi-tier app on 1 pod, it's a waste of idle resources
    * It's significantly easier to scale horizontally!

* When to use multiple containers in a pod:
    * Q1: Do they need to be run together or can they run on different hosts?
    * Q2: Are they a single whole or independent components?
    * Q3: Must they be scaled together or individually?

* A container shouldn't run multiple processes
* A pod shouldn't contain multiple containers that don't need to run on the same machine

## Creating Pods from YAML or JSON descriptors
* yaml and json formatting allows for easy version control for resource deployment

```
# Example Manifest
apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/created-by: ...
  creationTimestamp: 2016-03-18T12:37:50Z
  generateName: kubia-
  labels:
    run: kubia
  name: kubia-zxzij
  namespace: default
  resourceVersion: "294"
  selfLink: /api/v1/namespaces/default/pods/kubia-zxzij
  uid: 3a564dc0-ed06-11e5-ba3b-42010af00004
spec:
  containers:
  - image: luksa/kubia
    imagePullPolicy: IfNotPresent
    name: kubia
    ports:
    - containerPort: 8080
      protocol: TCP
    resources:
      requests:
        cpu: 100m
    terminationMessagePath: /dev/termination-log
    volumeMounts:
    - mountPath: /var/run/secrets/k8s.io/servacc
      name: default-token-kvcqa
      readOnly: true
  dnsPolicy: ClusterFirst
  nodeName: gke-kubia-e8fe08b8-node-txje
  restartPolicy: Always
  serviceAccount: default
  serviceAccountName: default
  terminationGracePeriodSeconds: 30
  volumes:
  - name: default-token-kvcqa
    secret:
      secretName: default-token-kvcqa
status:
  conditions:
  - lastProbeTime: null
    lastTransitionTime: null
    status: "True"
    type: Ready
  containerStatuses:
  - containerID: docker://f0276994322d247ba...
    image: luksa/kubia
    imageID: docker://4c325bcc6b40c110226b89fe...
    lastState: {}
    name: kubia
    ready: true
    restartCount: 0
    state:
      running:
        startedAt: 2016-03-18T12:46:05Z
  hostIP: 10.132.0.4
  phase: Running
  podIP: 10.0.2.3
  startTime: 2016-03-18T12:44:32Z
```

### Pod Definition Composition
* Kubernetes API version
* Resource Kind/Category
* Metadata
  * Name, namespace, labels, and other info about the pod
* Spec:
  * Description of the pod's content
* Status:
  * Current information about the running pod

* Containers usually log to the stanrd IO rather than just to log files


## Organizing Pods w/ Labels
* Labels: Arbitrary key:value pair which can be attached to any resource
* Generally used for small pieces of information
* Resources can have > 1 label so long as the labels are unique
* Labels can be modified without need for restarting the resource
* Labels can also be used to differentiate between different node requirements in a heterogenous infrasatructure
  * Ex if certain pods need access to a more powerful GPU or an SSD

* Label Selectors: Filters based on whether they include a specified label
* Label Selectors can select resources based on the following:
  * Does / Doesn't contain a label with a certain key
  * Contains a label with a certain key/value
  * Contains a label with a certain key but not with a certain value
* Label Selectors can also include multiple comma-separated criteria

## Organizing Pods w/ Annotations
* Also utilizes key:value pairs, but they aren't meant to hold identifying info
* Generally used for large pieces of information
  * Ex: descriptions for different resources
* Can also not be used to group objects vis-à-vi label selectors
* Typically will use an annotation over a field when doing alpha/beta work

## Organizing w/ Namespaces
* NOT THE SAME AS LINUX NAMESPACES
  * Linux namespaces isolate processes from each other
  * Kubernetes namespaces allows for a scope to group objec tnames
* Multiple namespaces allows you to split complex systems with numerous components into smaller groups
* Multiple namespaces though don't actually do any isolation of running objects!
